### 检测 gpu

In [ ]:
# 检查gpu状态
!nvidia-smi

In [ ]:
#@title STEP 0 连接 Google Drive
#@markdown #STEP 0
#@markdown ##连接 Google Drive
#@markdown ##Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title STEP 1 复制储存库并安裝必要的函式庫
#@markdown #STEP 1
#@markdown ##复制储存库并安裝必要的函式庫
#@markdown ##Clone repository & Install requirements lib
#@markdown ###
#@markdown ###选择训练版本
training_mode = "一般版本" # @param ["一般版本", "中文特化版"]
training_modes = ["一般版本", "中文特化版"]
if training_mode == training_modes[0]:
    !git clone https://github.com/fishaudio/Bert-VITS2.git
elif training_mode == training_modes[1]:
    !git clone https://github.com/fishaudio/Bert-VITS2.git --branch Extra-Fix
%cd ./Bert-VITS2/
!pip install -r requirements.txt
!pip install pyyaml

#下載 Bert 以及 wavlm 模型
!wget https://huggingface.co/microsoft/wavlm-base-plus/resolve/main/pytorch_model.bin?download=true -O slm/wavlm-base-plus/pytorch_model.bin

if training_mode == training_modes[0]:
    !rm -rf bert/chinese-roberta-wwm-ext-large
    !git clone https://huggingface.co/hfl/chinese-roberta-wwm-ext-large bert/chinese-roberta-wwm-ext-large
    !rm -rf bert/deberta-v2-large-japanese-char-wwm
    !git clone https://huggingface.co/ku-nlp/deberta-v2-large-japanese-char-wwm bert/deberta-v2-large-japanese-char-wwm
    !rm -rf bert/deberta-v3-large
    !git clone https://huggingface.co/microsoft/deberta-v3-large bert/deberta-v3-large

elif training_mode == training_modes[1]:
    !rm -rf bert/Erlangshen-MegatronBert-1.3B-Chinese
    !git clone https://huggingface.co/IDEA-CCNL/Erlangshen-MegatronBert-1.3B bert/Erlangshen-MegatronBert-1.3B-Chinese
    !rm -rf emotional/clap-htsat-fused
    !git clone https://huggingface.co/laion/clap-htsat-fused emotional/clap-htsat-fused
    !rm -rf emotional/wav2vec2-large-robust-12-ft-emotion-msp-dim
    !git clone https://huggingface.co/audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim emotional/wav2vec2-large-robust-12-ft-emotion-msp-dim
    !wget https://huggingface.co/ADT109119/G2PWModel-v2-onnx/resolve/main/g2pw.onnx?download=true -O g2pW/g2pW.onnx


上传数据集

In [ ]:
!unzip  /content/Bert-VITS2/data/lcc/lcc.zip  -d   /content/Bert-VITS2/data/lcc

# Bert-VITS2 数据预处理

数据准备：
将数据放置在 data 文件夹下，按照如下结构组织：

```
├── data
│   ├── {你的数据集名称}
│   │   ├── esd.list
│   │   ├── raw
│   │   │   ├── ****.wav
│   │   │   ├── ****.wav
│   │   │   ├── ...
```

其中，`raw` 文件夹下保存所有的音频文件，`esd.list` 文件为标签文本，格式为

```
****.wav|{说话人名}|{语言 ID}|{标签文本}\n
```

例如：
```
vo_ABDLQ001_1_paimon_02.wav|派蒙|ZH|没什么没什么，只是平时他总是站在这里，有点奇怪而已
noa_501_0001.wav|NOA|JP|そうだね、油断しないのはとても大事なことだと思う\n"
Albedo_vo_ABDLQ002_4_albedo_01.wav|Albedo|EN|Who are you? Why did you alarm them?
```

In [ ]:
#@title STEP 2.5 处理资料
#@markdown #STEP 2.5
#@markdown ##处理资料
#@markdown ##Process Data
from webui_preprocess import generate_config, resample, preprocess_text, bert_gen
batch_size = 8 # @param {type:"integer"}
data_dir = "lcc"  #@param {type:"string"}
print(generate_config(data_dir, batch_size))
print(resample(data_dir))
print(preprocess_text(data_dir))

#修改 config.yml
import yaml
def load_yaml(file_path):
    with open(file_path, 'r', encoding="utf-8") as file:
        data = yaml.load(file, Loader=yaml.FullLoader)
    return data
def save_yaml(data, file_path):
    with open(file_path, 'w', encoding="utf-8") as file:
        yaml.dump(data, file, default_flow_style=False)
# 載入 YAML 檔案
yaml_file_path = 'config.yml'
yaml_data = load_yaml(yaml_file_path)
# 修改 YAML 中的內容
yaml_data['dataset_path'] = f"data/{data_dir}"
yaml_data['train_ms']['config_path'] = "configs/config.json"
if training_mode == training_modes[1]:
    yaml_data['bert_gen']['num_processes'] = 1

# 儲存修改後的 YAML 檔案
save_yaml(yaml_data, yaml_file_path)
print(bert_gen(data_dir))

if training_mode == training_modes[1]:
    !python clap_gen.py --config data/{data_dir}/configs/config.json

In [ ]:
#@title STEP 2.6 自動儲存步數修改
#@markdown #STEP 2.6
#@markdown ##自動儲存步數修改

#@markdown ###每隔n步自動儲存
save_step = 200 # @param {type:"integer"}

import json

def load_json(file_path):
    with open(file_path, 'r', encoding="utf-8") as file:
        data = json.load(file)
    return data

def save_json(data, file_path):
    with open(file_path, 'w', encoding="utf-8") as file:
        json.dump(data, file, indent=2)

# 載入 JSON 檔案
json_file_path = f"data/{data_dir}/configs/config.json"
json_data = load_json(json_file_path)

# 修改 JSON 中的內容
json_data['train']['log_interval'] = save_step
json_data['train']['eval_interval'] = save_step

# 儲存修改後的 JSON 檔案
save_json(json_data, json_file_path)


# STEP 3 载入

模型
## 3-1、3-2 則一

In [ ]:
#@title STEP 3-1 下载预训练模型
if training_mode == training_modes[0]:
    !git clone https://huggingface.co/OedoSoldier/Bert-VITS2-2.3 data/{data_dir}/models
elif training_mode == training_modes[1]:
    !git clone https://huggingface.co/v3ucn/Bert-vits2-Extra-Pretrained_models data/{data_dir}/models

#STEP 4 开始训练

In [ ]:
import os
os.environ['TENSORBOARD_BINARY'] = '/usr/local/bin/tensorboard'
%reload_ext tensorboard
model_dir = f"./data/{data_dir}/models"
%tensorboard --logdir $model_dir

!torchrun --nproc_per_node=1 train_ms.py

#STEP 5 开始推理

In [ ]:
#@title STEP 5 推理测试
#@markdown #STEP 5
#@markdown ##推理测试

#@markdown ###
#@markdown ###推理所使用的模型步數
model_step = 2000 # @param {type:"integer"}
import yaml
def load_yaml(file_path):
    with open(file_path, 'r') as file:
        data = yaml.load(file, Loader=yaml.FullLoader)
    return data
def save_yaml(data, file_path):
    with open(file_path, 'w') as file:
        yaml.dump(data, file, default_flow_style=False)
# 載入 YAML 檔案
yaml_file_path = 'config.yml'
yaml_data = load_yaml(yaml_file_path)
# 修改 YAML 中的內容
yaml_data['dataset_path'] = f"data/{data_dir}"
yaml_data['webui']['share'] = "true"
yaml_data['webui']['model'] = f"models/G_{model_step}.pth"
yaml_data['webui']['config_path'] = "configs/config.json"
# 儲存修改後的 YAML 檔案
save_yaml(yaml_data, yaml_file_path)
!torchrun --nproc_per_node=1 webui.py